In [2]:
# 256*128*256列主
import numpy as np

n = 4
# matrix_C = matrix_A * matrix_B
data_a = np.random.randint(-5, 5, size=(16, ), dtype=int)
# print(data_a)
data_b = np.random.randint(-5, 5, size=(16, ), dtype=int)
matrix_A = data_a.reshape(4, 4)
matrix_B = data_b.reshape(4, 4)

print(matrix_A)
print(matrix_B)

[[ 3 -5 -5 -3]
 [ 1  1  4  3]
 [ 2  2  0 -5]
 [ 4 -3  0  3]]
[[ 2  1  0  0]
 [ 4  0 -3 -4]
 [ 4 -3  4  4]
 [-2  1  4  3]]


In [3]:
def tweak1(mat):
    matrix_C = mat.copy()
    for i in range(mat.shape[0]):
        for j in range(mat.shape[1]):
            matrix_C[i, j] = mat[i, (j+i) % mat.shape[1]]
    return matrix_C

def tweak1_inv(mat):
    matrix_C = mat.copy()
    for i in range(mat.shape[0]):
        for j in range(mat.shape[1]):
            matrix_C[i, j] = mat[i, (j-i) % mat.shape[1]]
    return matrix_C

def tweak2(mat):
    matrix_C = mat.copy()
    for i in range(mat.shape[0]):
        for j in range(mat.shape[1]):
            matrix_C[i, j] = mat[(j+i) % mat.shape[0], j]
    return matrix_C


# def tweak2_inv(mat):
#     matrix_C = mat.copy()
#     for i in range(mat.shape[0]):
#         for j in range(mat.shape[1]):
#             matrix_C[i, j] = mat[(i-j) % mat.shape[0], j]
#     return matrix_C

def tweak3(mat, k):
    matrix_C = mat.copy()
    for i in range(mat.shape[0]):
        for j in range(mat.shape[1]):
            matrix_C[i, j] = mat[i, (j+k) % mat.shape[1]]
    return matrix_C

def tweak4(mat, k):
    matrix_C = mat.copy()
    for i in range(mat.shape[0]):
        for j in range(mat.shape[1]):
            matrix_C[i, j] = mat[(i+k) % mat.shape[0], j]
    return matrix_C

# temp1 = tweak1(matrix_A)
# temp2 = tweak2(matrix_B)

# n = matrix_A.shape[0]
# d = matrix_A.shape[1]

# plain = np.zeros(16).reshape(4, 4)

# print(matrix_A @ matrix_B)
 
# for i in range(n):
#     plain += tweak3(temp1, i) * tweak4(temp2, i)
# print(plain)

In [4]:
# print(matrix_A)
# print(matrix_B)

mask1 = np.array([1 if (i % n == 0) else 0 for i in range(n*n)]).reshape(n, n)
mask2 = np.array([1 if (i % n == 1) else 0 for i in range(n*n)]).reshape(n, n)
mask3 = np.array([1 if (i % n == 2) else 0 for i in range(n*n)]).reshape(n, n)
mask4 = np.array([1 if (i % n == 3) else 0 for i in range(n*n)]).reshape(n, n)

def rot_left(mat, k):
    return tweak3(mat, k)
def rot_up(mat, k):
    return tweak4(mat, k)

rot_B = []
for i in range(n):
    temp = np.zeros(n*n).reshape(n, n)
    for j in range(n):
        temp[j] = matrix_B[(j+i)%n]
    rot_B.append(matrix_A * temp)

# phase 1
rot_B[0] += rot_left(rot_B[0], 2)
rot_B[0] = rot_B[0] * (mask1 + mask2)
# print(rot_B[0])

# print(rot_B[1])
rot_B[1] += rot_left(rot_B[1], 2)
rot_B[1] *= (mask1 + mask2)
# print(rot_B[1])

# print(rot_B[2])
rot_B[2] += rot_left(rot_B[2], -2)
rot_B[2] *= (mask3 + mask4)
# print(rot_B[2])


rot_B[3] += rot_left(rot_B[3], -2)
rot_B[3] *= (mask3 + mask4)

rot_B[0] += rot_B[2]
# print(rot_B[0])
rot_B[1] += rot_B[3]


rot_B[0] += rot_left(rot_B[0], 1)
rot_B[0] *= (mask1 + mask3)

rot_B[1] += rot_left(rot_B[1], -1)
rot_B[1] *= (mask2 + mask4)


# print(tweak1(tweak1_inv(rot_B[0])))


# mat_C = tweak1_inv(rot_B[0])
# mat_D = matrix_B.copy()


# # temp1 = tweak1(mat_C)
# temp1 = rot_B[0]
# temp2 = tweak2(mat_D)

# plain = np.zeros(16).reshape(4, 4)

# print(mat_C @ mat_D)
 
# for i in range(n):
#     plain += tweak3(temp1, i) * tweak4(temp2, i)
# print(plain)
rot_B[0] += rot_B[1]
print((rot_B[0]))
print(tweak1_inv(rot_B[0]))
print(matrix_A @ matrix_B.T)

[[  1.  39.  -5. -40.]
 [-20.  29.  24.   3.]
 [-18. -17.   6.  28.]
 [ -2.   5.   4.  37.]]
[[  1.  39.  -5. -40.]
 [  3. -20.  29.  24.]
 [  6.  28. -18. -17.]
 [  5.   4.  37.  -2.]]
[[  1  39  -5 -40]
 [  3 -20  29  24]
 [  6  28 -18 -17]
 [  5   4  37  -2]]


In [5]:
import numpy as np

token_len = 128
head = 4
d = 64
column_num = head*d

# 产生S
data_S = []
for i in range(head):
    data_S.append(np.random.uniform(-1, 1, size=(token_len * token_len)).reshape(token_len,token_len))

cipher_S = []
for i in range(int(token_len/d)):
    cipher_S_i = data_S[0][:,i*d:i*d+d]
    for j in range(1,head):
        cipher_S_i = np.concatenate((cipher_S_i, data_S[j][:,i*d:i*d+d]), axis = 1)
    cipher_S.append(cipher_S_i)
print(f"len cipher_S {len(cipher_S)}")

#S_sigma
data_S_sigma = []
for i in range(head):
    data_S_sigma.append(tweak1(data_S[i]))

cipher_S_sigma = []
for i in range(int(token_len/d)):
    cipher_S_sigma_i = data_S_sigma[0][:,i*d:i*d+d]
    for j in range(1,head):
        cipher_S_sigma_i = np.concatenate((cipher_S_sigma_i, data_S_sigma[j][:,i*d:i*d+d]), axis = 1)
    cipher_S_sigma.append(cipher_S_sigma_i)

# 写入S
for i in range(len(cipher_S_sigma)):
    with open("data/sigma_S_{i}.txt", 'w') as f:
        for i in range(len(cipher_S_sigma)):
            to_write = cipher_S_sigma[i].reshape(-1)
            f.write(' '.join(str(x) for x in to_write))

# 产生v
data_V = []
for i in range(head):
    data_V.append(np.random.uniform(-1, 1, size=(token_len * d)).reshape(token_len,d))

cipher_V = []
for i in range(1):
    cipher_V_i = data_S[0][:,i*d:i*d+d]
    for j in range(1,head):
        cipher_V_i = np.concatenate((cipher_V_i, data_V[j][:,i*d:i*d+d]), axis = 1)
    cipher_V.append(cipher_V_i)
print(f"len cipher_V {len(cipher_V)}")

# 写入V
with open("data/V.txt", 'w') as f:
    V_write = cipher_V[0].reshape(-1)
    f.write(' '.join(str(x) for x in V_write))

# tau V
data_V_tau = []
for i in range(head):
    data_V_tau.append(tweak1(data_V[i]))

cipher_V_tau = []
for i in range(1):
    cipher_V_tau_i = data_V_tau[0][:,i*d:i*d+d]
    for j in range(1,head):
        cipher_V_tau_i = np.concatenate((cipher_V_tau_i, data_V_tau[j][:,i*d:i*d+d]), axis = 1)
    cipher_V_tau.append(cipher_V_tau_i)

# 写入tau_V
with open("data/tau_V.txt", 'w') as f:
    to_write = cipher_V_tau[0].reshape(-1)
    f.write(' '.join(str(x) for x in to_write))



# mul_V_res
data_res = []
for i in range(head):
    data_res.append(data_S[i] @ data_V[i])
cipher_res = []
cipher_res_i = data_res[0]
for i in range(1,head):
    cipher_res_i = np.concatenate((cipher_res_i, data_res[i]), axis = 1)
cipher_res.append(cipher_res_i)
# write res
with open("data/mul_V_res.txt", 'w') as f:
    to_write = cipher_res[0].reshape(-1)
    f.write(' '.join(str(x) for x in to_write))



len cipher_S 2
len cipher_V 1


In [1]:
import numpy as np

token_len = 128
head = 4
d = 64
column_num = head*d
N = token_len * column_num

data_V = np.random.randint(0,100,(N))
linear_mat = np.zeros((N,N))

lis = []
for i in range(token_len):
    for j in range(d):
        for k in range(head):
            before = i*column_num + k *d +j
            target = ((i+j)%token_len) * column_num  + k * d +j
            a = (target-before)%N
            if a not in lis:
                lis.append(a)
            linear_mat[before][target] = 1
print("diag_num")
print(len(lis))

data_V = data_V.reshape(N,1)
res = linear_mat @ data_V
res = res.reshape(token_len, column_num)

data_V = data_V.reshape(token_len, column_num)
# print("data_V is")
# print(data_V)
# print("res is")
# print(res)

tau_data_V_multi_head = []
for i in range(head):
    tau_data_V_multi_head.append(tweak2(data_V[:,d*i:d*i+d]))
tau_data_V = np.concatenate(tau_data_V_multi_head, axis = 1)
for i in range(token_len):
    for j in range(column_num):
        assert tau_data_V[i][j] == res[i][j]

# print("tau_data_V is")
# print(tau_data_V)
diag_num = 0
for j in range(N):
    one = 0
    for i in range(N):
        if(linear_mat[i][(i+j)%column_num]==1):
            one = 1
            print(i,end = ' ')
            # break
    if (one):
        print(f"\n{j}")
        diag_num = diag_num + 1
    
print(diag_num)

diag_num
64


NameError: name 'tweak2' is not defined

In [2]:
B_lis = [[(i*512+j) for i in range(128)] for j in range(512)]
# print(B_lis[0])

def bitReverse(a, bit_len):
    bin_str = bin(a)[2:].zfill(bit_len)
    
    # 反转二进制字符串
    reversed_bin_str = bin_str[::-1]
    
    # 将反转后的二进制字符串转换为整数
    reversed_int = int(reversed_bin_str, 2)
    
    return reversed_int
for lis in B_lis:
    for j in range(len(lis)):
        lis[j] = bitReverse(lis[j], 15)
print(B_lis[0])
print(B_lis[1])


[0, 32, 16, 48, 8, 40, 24, 56, 4, 36, 20, 52, 12, 44, 28, 60, 2, 34, 18, 50, 10, 42, 26, 58, 6, 38, 22, 54, 14, 46, 30, 62, 1, 33, 17, 49, 9, 41, 25, 57, 5, 37, 21, 53, 13, 45, 29, 61, 3, 35, 19, 51, 11, 43, 27, 59, 7, 39, 23, 55, 15, 47, 31, 63, 1, 65, 33, 97, 17, 81, 49, 113, 9, 73, 41, 105, 25, 89, 57, 121, 5, 69, 37, 101, 21, 85, 53, 117, 13, 77, 45, 109, 29, 93, 61, 125, 3, 67, 35, 99, 19, 83, 51, 115, 11, 75, 43, 107, 27, 91, 59, 123, 7, 71, 39, 103, 23, 87, 55, 119, 15, 79, 47, 111, 31, 95, 63, 127]
[16384, 16416, 16400, 16432, 16392, 16424, 16408, 16440, 16388, 16420, 16404, 16436, 16396, 16428, 16412, 16444, 16386, 16418, 16402, 16434, 16394, 16426, 16410, 16442, 16390, 16422, 16406, 16438, 16398, 16430, 16414, 16446, 16385, 16417, 16401, 16433, 16393, 16425, 16409, 16441, 16389, 16421, 16405, 16437, 16397, 16429, 16413, 16445, 16387, 16419, 16403, 16435, 16395, 16427, 16411, 16443, 16391, 16423, 16407, 16439, 16399, 16431, 16415, 16447, 32769, 32833, 32801, 32865, 32785, 3284

In [13]:
N = 2**13
logN = 5
matR = np.zeros((N,N))
for i in range(N):
    matR[i][bitReverse(i, logN)] = 1
print(matR)


acc = 0
for i in range(N):
    flag = 0
    for j in range(N):
        if matR[(i+j)%N][j] == 1:
            flag = 1
            break
    if flag == 1:
        acc += 1
        print(i)
print(acc)

[[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
0
4
6
8
9
10
12
13
14
15
16
17
18
20
21
24
26
27
28
29
30
31
32
33
34
35
36
37
40
41
42
45
48
49
51
52
54
56
57
58
60
61
62
63
65
66
68
69
70
72
73
74
75
77
80
81
82
84
85
90
91
93
96
98
99
101
102
104
105
107
108
112
114
116
117
119
120
122
124
125
126
127
129
130
132
133
135
136
138
140
141
144
146
147
148
149
150
153
154
155
157
161
162
163
164
165
168
169
170
171
173
180
181
182
185
186
189
195
196
197
198
201
202
204
205
209
210
213
214
216
217
219
225
228
231
234
237
238
240
241
248
252
254
255
257
258
260
261
264
266
269
270
273
276
279
280
282
285
291
292
294
297
298
300
301
306
309
310
313
315
321
322
324
325
326
328
329
330
331
333
336
338
339
340
341
342
345
346
347
349
357
360
362
364
365
370
371
372
373
378
381
387
390
392
393
394
396
397
402
403
404
405
408
410
413
417
418
420
421
426
427
428
429
434
437
438
439
441
44